## Google Cloud storage

> Here the data is stored in **buckets**. 

> They are **scalable** and are **charged based on the size of contents in bucket**

> Commonly used for storing **multimedia** or **storing/accessing data for your web app** or **Serving static websites**.

### There are different storage classes for different usecases

>1) **Multi-regional** : If the data in bucket is frequently accessed from around the globe (USD 0.025/GB/month)

>2) **Regional** : If the data is accessed frequently only from a particular region (USD 0.02/GB/month)

>3) **Nearline**: If data is not frequently accessed. Mostly used for backup data (USD 0.01/GB/month). 
But has access cost of USD 0.01 per GB and charged for minimum 30 days

>4) **coldline**: If data is not frequently accessed but required high latency.(USD 0.007/GB/month) Used when restore time of back up data should be quicker. Access cost is more than that of nearline (USD 0.05/GB)

### Creating a bucket

> Done via **GCP console** under **Storage > storage**

> The **name of the bucket** should be **globally unique**

> Creating in **command line** using **[gsutil](https://cloud.google.com/storage/docs/gsutil)** - tool to access cloud storage
```bash
gsutil mb -p [PROJECT_NAME] -c [STORAGE_CLASS] -l [BUCKET_LOCATION] gs://[BUCKET_NAME]/
```

### List all buckets in a project

>```bash
gsutil ls
(or)
gsutil list
```
> For detailed information
```bash
gsutil list -L -b gs://[BUCKET_NAME]
```

### To **list files inside a bucket**

>```bash
gsutil ls gs://[BUCKET_NAME]
```

### Checking size of the bucket

>This is done via commandline. This command displays the size of all buckets under a project
```bash
gsutil du -s -h
```
**Note:** -h displays the size in KB/MB/GB

>To display size of single bucket
```bash
gsutil du -s -h gs://[BUCKET_NAME]/
```

### Uploading files

> This can be done through the console or through **gsutil**
```bash
gsutil cp [FILE_PATH] gs://[BUCKET_NAME]
```

> For directories,
```bash
gsutil cp -r [DIR_PATH] gs://[BUCKET_NAME]
```

> To synchronize local directory with a cloud directory directory
```bash
gsutil rsync -r [DIR_NAME] gs://[BUCKET_NAME]/[DIR_NAME]
```

> To copy files between buckets
```bash
gsutil cp -r -p gs://[BUCKET_NAME_1]/* gs://[BUCKET_NAME_2]
```
**-p** copies over **permission** of all **files**

### Hosting a static webpage

> **Upload** HTML file(s). By default, **permission** is **not public**

> Click the three dots next to file name and **edit permissions**.

>> Select **User** for the Entity.

>> Enter **allUsers** for the Name.
    
>> Select **Reader** for the Access.

> Through **[gsutil acl](https://cloud.google.com/storage/docs/gsutil/commands/acl)**
```bash
gsutil acl ch -u AllUsers:R gs://[BUCKET_NAME]/[FILE_NAME]
```

> Your file will now be **public** and there will be an **icon which will redirect you to the URL**

>> The URL is of form:
```bash
https://storage.googleapis.com/[BUCKER_NAME]/[FILE_NAME]
```

### Setting lifecycle

> The lifecycle configuration specifies that all objects in a bucket that are more than certain period old will be deleted automatically or changed to nearline/coldline storage class

> They are useful when we want to **manage log files**

> This can be applied only to whole bucket and not to individual files

> This can be done easily through **console** or through command line as follows
```bash
gsutil lifecycle set config.json gs://[BUCKET_NAME]
```
>>**config.json**
```json
{
  "rule":
  [
    {
      "action": {"type": "Delete"},
      "condition": {"age": 365} #Files more than 365 days will be deleted
    }
  ]
}
```

> To **get** lifecycle settings for a bucket
```bash
gsutil lifecycle get gs://[BUCKET_NAME]
```

### Transfer service

> Available under **Storage > Transfer**

> Used to transfer from other online repositories like **AWS** or from some URL. They can also be used for transfers with GCP. They have additional functionalities like, **scheduling transfers everyday at certain time**, **Deleting files from source bucket after transfer** etc

> Prefer **gsutil** for transfers from **local machine** or **transfers within GCP**

### Service accounts 

**Provide separate folders in cloud for every VM instance/ local machines **

A service account is a special Google account that belongs to your application or a virtual machine (VM), instead of to an individual end user. Your application uses the service account to call the Google API of a service, so that the users aren't directly involved.

For example, a Compute Engine VM may run as a service account, and that account can be given permissions to access the resources it needs. This way the service account is the identity of the service, and the service account's permissions control which resources the service can access.

> Go to **IAM&Admin > service accounts** and create a service account for your project

> Specify the **access** - Editor/Viewer/Owner/etc

> Generate a **json key** for that service account

> In your new VM instances, place that **json key** and **autherize gcloud**
```bash
gcloud auth activate-service-account --key-file [JSON_KEY_FILE]
gcloud init
```

> If the permission was **Viewer**, **gsutil** can only **view files** in the cloud. **Cannot create, start or stop VM instances**. If the permission was **editor**, gsutil can **upload** new files to cloud and set permissions.
```bash
gsutil cp newfile gs://[BUCKET_NAME]
gsutil acl set private gs://[BUCKET_NAME]/new_file
```
This will make the file accessible only from this VM instance (service account)

> Thus **each service accounts (VM instances/local computers) can hold private folders in cloud**

**Modifying permissions of service account**

Permissions can be **changed** or  more permissions can be **added** under **IAM** overview 

**To access storage buckets from different project**

> VM instance in **project-1**

> Storage bucket in **project-2**

> Create **service account in project 2** with permission **storage viewer** and generate **json key**

> Log on to VM instance in **project-1** and use **gcloud auth** with the key

> In the Vm instance **gcloud init** with the cross-project service account and set the project as **project-2**

> Now we can access the storage in **project-2** with **gsutils** from the VM instance in **project-1**

### Versioning

> Enable versioning for cloud bucket

>```bash
gsutil versioning set [on|off] gs://[BUCKET_NAME]
```
>  Now we can get a **version number for each file** in the bucket using the command. Everytime we replace the file in the cloud, all the old versions will be available
```bash
gsutil ls -a gs://[BUCKET_NAME]
```
>>```bash
setup.txt#150987263
```

>```bash
gsutil cp setup.txt gs://[BUCKET_NAME] (Upload another file)
gsutil ls -a gs://[BUCKET_NAME]
```
>>```bash
setup.txt#150987263 (Old version)
setup.txt#151098234
```